In [1]:
import numpy as np 
import pandas as pd 
import chardet
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation

In [5]:
def find_encoding(fname):
    r_file = open(fname, 'rb').read()
    result = chardet.detect(r_file)
    charenc = result['encoding']
    return charenc

In [6]:
my_encoding = find_encoding('data/wines.csv')
my_encoding

'ISO-8859-1'

In [16]:
wines_col = ['wine_id', 'name', 'type', 'country', 'region', 'alcohol_content', 'producer', 'service', 'volume', 'vintage']
wines = pd.read_csv('data/wines.csv', sep=';', names=wines_col, encoding=my_encoding, engine='python')
wines.head()

,wine_id,name,type,country,region,alcohol_content,producer,service,volume,vintage
0,1,Expedicion Single Vineyard Selection Cabernet ...,Tinto,Chile,Vale Central,13.0,Finca Patagonia,17,750,2019.0
1,2,Duo R One Ronaldinho Brunello di Montalcino DO...,Tinto,Italia,Toscana,15.0,Fabio Cordela,18,750,2014.0
2,3,Jules Desvignes Pinot Noir 2016,Tinto,França,Languedoc-Roussillon,12.5,Domaine Du Pere Guillot,16,750,2016.0
3,4,Anciano Gran Reserva 10 years Valdepeñas D.O. ...,Tinto,Espanha,Valdepenas,13.0,Anciano,17,750,2007.0
4,5,Mil Raices Selección Especial Cabernet Sauvign...,Tinto,Chile,NaN,13.0,Mil Raices,17,750,NaN


In [18]:
matrix_col = ['user_id', 'wine_id', 'rating']
matrix = pd.read_csv('data/matrix.csv', sep=';', names=matrix_col, encoding=my_encoding, engine='python')
matrix.head()

,user_id,wine_id,rating
0,10,81,1
1,7,52,4
2,3,34,2
3,1,12,1
4,4,61,1
